## Trace Example: fibonacci computation

In [2]:
# Initialize the finite field (e.g., GF(17))
p = 101
F = GF(p)
trace = [(F(1), F(1))]  # Initial state (a₀, a₁)

# Generate the computation trace
steps = 4  # Produces 5 rows (steps 0 to 4)
for _ in range(steps):
    a, b = trace[-1]
    next_a = b
    next_b = a + b
    trace.append((next_a, next_b))

print("Computation Trace:")
for i, (a, b) in enumerate(trace):
    print(f"Step {i}: ({a}, {b})")

Computation Trace:
Step 0: (1, 1)
Step 1: (1, 2)
Step 2: (2, 3)
Step 3: (3, 5)
Step 4: (5, 8)


## Calculate Boundary Quotients

In [3]:
R.<x> = PolynomialRing(F)
omega = F.multiplicative_generator() ^ ((p-1) // steps)
domain = [omega ^ i for i in range(steps)]
Z = prod([(x-root) for root in domain])
assert Z == (x^4 - 1)


# Interpolate polynomials for columns A and B
A_values = [row[0] for row in trace]
B_values = [row[1] for row in trace]

# trace polynomials
A_poly = R.lagrange_polynomial(zip(domain, A_values))
B_poly = R.lagrange_polynomial(zip(domain, B_values))

print("A(x) =", A_poly)
print("B(x) =", B_poly)

# substract boundary polys to trace polys
boundary_a, boundary_b = trace[0] 
boundaryA_poly = A_poly - R.lagrange_polynomial(zip([domain[0]],[boundary_a]))
boundaryB_poly = B_poly - R.lagrange_polynomial(zip([domain[0]],[boundary_b]))

(boundaryA_quotients, rest) = boundaryA_poly.quo_rem((x-domain[0]))
assert rest == 0
(boundaryB_quotients, rest) = boundaryB_poly.quo_rem((x-domain[0]))
assert rest == 0

print("bound_quoA(x) =", boundaryA_quotients)
print("bound_quoB(x) =", boundaryB_quotients)

A(x) = 20*x^3 + 25*x^2 + 30*x + 27
B(x) = 93*x^3 + 75*x^2 + 7*x + 28
bound_quoA(x) = 20*x^2 + 45*x + 75
bound_quoB(x) = 93*x^2 + 67*x + 74


## Calculate transition quotiones

In [8]:
## transition polynomials
NEXT_poly= A_poly(omega * x) - B_poly(x)
SUM_poly = B_poly(omega * x) - (A_poly(x) + B_poly(x))


TRANSITION_poly = NEXT_poly(x) + SUM_poly(x)

for w in domain[:-1]:
    assert NEXT_poly(w) == 0 and SUM_poly(w) == 0


68*x^3 + 27*x^2 + 33*x + 74 9*x^3 + 44*x^2 + 33*x + 23


AssertionError: 

## Verification

In [5]:
# verification
Z = (x^4 - 1)

# verify boundaries
# 2. The verifier checks that tw(X) evaluates to e in οi for all (i,w,e)∈B.
assert  boundaryA_poly.quo_rem((x-domain[0]))[1] == 0 and boundaryA_poly.quo_rem((x-domain[0]))[1] == 0

# transition constrains
# 5. The verifier checks that the transition polynomials c(X) evaluate to zero in {οi|i∈{0,…,T−2}}
Z /= (x - domain[-1])
assert NEXT_poly.quo_rem(Z)[1] == 0 and SUM_poly.quo_rem(Z)[1] == 0

